# Setup

In [6]:
!whoami

azureuser


In [7]:
!pip install tensorflow-gpu==2.3.0

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.5/320.5 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 80.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 120.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 104.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.0/459.0 kB 41.2 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.1
    Uninstalling numpy-1.23.1:
      Successfully uninstalled numpy-1.23.1
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled 

In [ ]:
%%capture
!pip install tensorflow[and-cuda]

In [3]:
%%capture
!pip install numpy==1.23.1 Pillow==9.2.0 importlib-metadata==4.11.3 matplotlib==3.5.2 scikit-learn==1.1.1 tensorboard==2.9.1 jupyter==1.0.0 ipykernel==6.9.1 seaborn==0.13.2

In [13]:
import sys
import os

# Add the project directory to the Python path
sys.path.append('/home/azureuser/projects/opencampus-preconditioner-ai-project')

# Change the working directory to the project directory
os.chdir('/home/azureuser/projects/opencampus-preconditioner-ai-project')
print(os.getcwd())  # Verify the change

FileNotFoundError: [Errno 2] No such file or directory: '/home/azureuser/opencampus-preconditioner-ai-project'

In [4]:
# Check wd
current_base_path = os.getcwd()
print(f"Path to current working directory: {current_base_path}")

import os
import scipy
import importlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from scipy.sparse.linalg import gmres
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# import matrixlib
import matrixlib.io
import matrixlib.util
import matrixlib.plot
import matrixlib.generate
from matrixlib.metadata import MatrixMetadata

#import CNN
import CNN.models
import CNN.training
import CNN.evaluation

# import solvers
#import solvers.gmres


Path to current working directory: /home/azureuser


2024-06-14 15:19:07.735161: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-14 15:19:07.738995: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-06-14 15:19:07.739006: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


ModuleNotFoundError: No module named 'matrixlib'

In [14]:
tf.config.list_physical_devices('GPU') 

[]

# Generate Synthetic Data
Generate a set of `n` square symmetrical and positive semi matrices of dimensions `MATRIX_DIM` to RAM.

## Define Constants
The following constants are used throughout the rest of the notebook.

In [9]:
MATRIX_DIM = 64
NUMBER_OF_MATRICES = 1000

AVERAGE_BLOCK_SIZE = 10
BLOCK_SIZE_STD_DEV = 0.66

NOISE_BACKGROUND_DENSITY_RANGE = (0.3, 0.5)
NOISE_BACKGROUND_VALUE_RANGE = (0.0, 0.5)

NOISE_BLOCK_GAP_CHANCE = 0.5
NOISE_BLOCK_SIZE_RANGE = (3, 32)
NOISE_BLOCK_DENSITY_RANGE = (0.3, 0.5)
NOISE_BLOCK_VALUE_RANGE = (0.0, 1.0)

TRUE_BLOCK_GAP_CHANCE = 0.0
TRUE_BLOCK_SIZE_RANGE = (2, 32)
TRUE_BLOCK_DENSITY_RANGE = (0.5, 0.7)
TRUE_BLOCK_VALUE_RANGE = (0.0, 1.0)

DIAGONAL_BAND_RADIUS = 10

RNG_SEED = 42

In [10]:
def generate_matrices(number_of_matrices: int, dimension: int) -> (np.ndarray, np.ndarray):
    generated_metadata: MatrixMetadata = MatrixMetadata(NUMBER_OF_MATRICES, MATRIX_DIM)
    generated_matrices: np.array = matrixlib.generate.__init_zero_matrices(number_of_matrices, dimension)

    # add background noise to the generated matrices
    matrixlib.generate.add_noise(
        generated_matrices,
        generated_metadata.noise_background_density,
        NOISE_BACKGROUND_DENSITY_RANGE,
        NOISE_BACKGROUND_VALUE_RANGE
    )

    # add noise blocks with higher values to the diagonal of the matrix
    noise_block_sizes = matrixlib.generate.add_blocks(
        generated_matrices,
        None,
        generated_metadata.noise_block_density,
        NOISE_BLOCK_DENSITY_RANGE,
        NOISE_BLOCK_VALUE_RANGE,
        NOISE_BLOCK_SIZE_RANGE,
        NOISE_BLOCK_GAP_CHANCE,
        AVERAGE_BLOCK_SIZE,
        BLOCK_SIZE_STD_DEV,
    )

    # add 'real' blocks to the diagonal of the matrix without gaps
    true_block_sizes = matrixlib.generate.add_blocks(
        generated_matrices,
        generated_metadata.block_starts,
        generated_metadata.true_block_density,
        TRUE_BLOCK_DENSITY_RANGE,
        TRUE_BLOCK_VALUE_RANGE,
        TRUE_BLOCK_SIZE_RANGE,
        TRUE_BLOCK_GAP_CHANCE,
        AVERAGE_BLOCK_SIZE,
        BLOCK_SIZE_STD_DEV,
    )

    return generated_matrices, generated_metadata

In [11]:
generated_matrices, metadata = generate_matrices(NUMBER_OF_MATRICES, MATRIX_DIM)
matrices = generated_matrices.reshape(NUMBER_OF_MATRICES, MATRIX_DIM, MATRIX_DIM, 1)
labels = metadata.block_starts

print(matrices.shape)

# Create dataset from matrices and labels
dataset = tf.data.Dataset.from_tensor_slices((matrices, labels))

# Split the dataset
train_size = int(0.8 * NUMBER_OF_MATRICES)
val_size = int(0.1 * NUMBER_OF_MATRICES)
test_size = NUMBER_OF_MATRICES - train_size - val_size

train_dataset = dataset.take(train_size).shuffle(buffer_size=10)
val_dataset = dataset.skip(train_size).take(val_size)
test_dataset = dataset.skip(train_size + val_size)

print(f"Train size: {train_size}, Val size: {val_size}, Test size: {test_size}")


NameError: name 'MatrixMetadata' is not defined

In [ ]:
# Verfiy Shape
print(train_dataset)
print(val_dataset)
print(test_dataset)

# for element in val_dataset:
#    print(element)

# Training

In [ ]:
# reload the library when changes were made to it
importlib.reload(matrixlib.io)
importlib.reload(matrixlib.util)
importlib.reload(matrixlib.plot)
importlib.reload(matrixlib.generate)
#importlib.reload(CNN.dataloader)

importlib.reload(CNN.models)
importlib.reload(CNN.training)
importlib.reload(CNN.evaluation)
importlib.reload(matrixlib.generate)

importlib.reload(solvers.gmres)
importlib.reload(solvers.blockjacobi)

In [ ]:
# Define parameters
loss_fn = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.000001)
num_epochs = 100
log_dir = "runs/matrix_cnn_experiment"

# Check log files
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
print("Files in log directory:", os.listdir(log_dir))

In [ ]:
# Compile Model
model = CNN.models.Baseline(input_shape=(MATRIX_DIM, MATRIX_DIM, 1))
model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=['accuracy'])

# Start Training Loop
trained_model, train_losses, val_losses = CNN.training.train_model(
    model=model,
    train_dataset=train_dataset.batch(8),  # Adjust when connected to vm
    val_dataset=val_dataset.batch(8),
    loss_fn=loss_fn, 
    optimizer=optimizer,
    num_epochs=num_epochs,
    log_dir=log_dir
)


In [ ]:
# Plot losses
def plot_losses(train_losses, val_losses):
    """
    Plots the training and validation losses.

    Args:
        train_losses (list of float): List of training losses per epoch.
        val_losses (list of float): List of validation losses per epoch.
    """
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss Over Epochs')
    plt.legend()
    plt.grid(True)
    plt.show()

plot_losses(train_losses, val_losses)

# Evaluation

In [ ]:
# Restore the model weights from the checkpoint
trained_model = CNN.models.Baseline(input_shape=(MATRIX_DIM, MATRIX_DIM, 1))

checkpoint_path = "runs/matrix_cnn_experiment/best_model_checkpoint"
checkpoint = tf.train.Checkpoint(model=trained_model)
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_path)).expect_partial()

In [ ]:
# Evaluate the model on the test set
evaluation_results = CNN.evaluation.evaluate_model(
    model=trained_model,
    test_dataset=test_dataset.batch(8),
    loss_fn=loss_fn
)

# Prediction

In [ ]:
# Extract test matrices and labels from dataset
test_matrices = generated_matrices[train_size + val_size:]
test_labels = metadata.block_starts[train_size + val_size:]

# Make Predictions
test_predictions = trained_model.predict(test_dataset.batch(8))

# Convert to Binary
threshold = 0.5
binary_predictions = (test_predictions >= threshold).astype(int)

# Calculate Metrics
overall_accuracy = (binary_predictions == test_labels).mean()
print("Overall Accuracy:")
print(overall_accuracy)

# Create Block Jacobi Preconditioner

In [ ]:
print(test_matrices.shape)
print(binary_predictions.shape)

In [ ]:
def block_jacobi_preconditioner_from_predictions(A, binary_predictions):
    """
    Calculates the block Jacobi preconditioner for each 2D matrix in a 3D array using binary predictions for block starts.

    Args:
        A: NumPy array of shape (n, m, m) representing n square matrices of size m x m.
        binary_predictions: NumPy array of shape (n, m) where each row indicates block starts with 1s.

    Returns:
        A NumPy array of the same shape as A representing the block diagonal preconditioner for each 2D matrix.
    """
    n, m, _ = A.shape
    
    # Initialize the preconditioner array
    prec = np.zeros_like(A)
    
    for k in range(n):
        # Find block start indices from binary predictions
        block_starts = np.where(binary_predictions[k] == 1)[0]
        
        # Ensure to include the end of the matrix as a block end
        block_starts = np.append(block_starts, m)
        
        # Extract diagonal blocks
        blocks = []
        for i in range(len(block_starts) - 1):
            start = block_starts[i]
            end = block_starts[i + 1]
            blocks.append(A[k, start:end, start:end])
        
        # Invert diagonal blocks if they are invertible
        inv_blocks = []
        for block in blocks:
            try:
                inv_block = np.linalg.inv(block)
            except np.linalg.LinAlgError:
                # Use pseudoinverse if the block is singular
                inv_block = np.linalg.pinv(block)
            inv_blocks.append(inv_block)
        
        # Create block diagonal matrix for the k-th matrix
        for i, block in enumerate(inv_blocks):
            start = block_starts[i]
            end = block_starts[i + 1]
            prec[k, start:end, start:end] = block
    
    return prec

In [ ]:
# Create preconditioner
prec = block_jacobi_preconditioner_from_predictions(test_matrices, binary_predictions)

print("Original Matrix A (first instance):")
print(A[0])
print("\nBlock Jacobi Preconditioner (first instance):")
print(prec[0])

# GMRES Solver

## Check Condition Number before Preconditioning

In [ ]:
print(test_labels.shape)
print(test_labels[0])

In [ ]:
def check_condition_number(matrix):
    try:
        condition_number = np.linalg.cond(matrix)
    except np.linalg.LinAlgError:
        condition_number = float('inf')
    return condition_number

check_condition_number(test_matrices[0])


In [ ]:

# Convert test_labels to integer indices
block_starts = np.where(test_labels[0] == 1)[0]

# Check condition numbers
for i, matrix in enumerate(test_matrices):
    condition_number = check_condition_number(matrix)
    print(f"Condition number of matrix {i}: {condition_number}")
    block_starts = np.where(binary_predictions[i] == 1)[0]
    block_starts = np.append(block_starts, matrix.shape[0])
    for j in range(len(block_starts) - 1):
        start = block_starts[j]
        end = block_starts[j + 1]
        block = matrix[start:end, start:end]
        block_condition_number = check_condition_number(block)
        print(f"Condition number of block {j} of matrix {i}: {block_condition_number}")
        prec_block = prec[i, start:end, start:end]
        prec_block_condition_number = check_condition_number(prec_block)
        print(f"Condition number of preconditioner block {j} of matrix {i}: {prec_block_condition_number}")

print("Original Matrix A (first instance):")
print(test_matrices[0])
print("\nBlock Jacobi Preconditioner (first instance):")
print(prec[0])

In [ ]:
# Run GMRES solver on unconditioned matrices
from scipy.sparse.linalg import gmres

# Define parameters
maxit = 100
tol = 1e-5

solutions_no_preconditioner = []  # List to store solution vectors for all matrices


for i, matrix in enumerate(test_matrices):
    # Convert matrix to sparse format
    A_sparse = scipy.sparse.csr_matrix(matrix)

    # Define right-hand side as a vector of ones
    b = np.ones(matrix.shape[0])

    # Extract the corresponding preconditioner for this matrix
    preconditioner_matrix = scipy.sparse.csr_matrix(prec[i])

    # Run GMRES solver
    x, iteration_number = gmres(A_sparse, b, M=preconditioner_matrix, tol=tol, maxiter=maxit)
    solutions_no_preconditioner.append(x)
    print(f"Matrix {i} solved in {iteration_number}")




In [ ]:
solutions_with_preconditioner = []  

for i, matrix in enumerate(test_matrices):
    # Convert matrix to sparse format
    A_sparse = scipy.sparse.csr_matrix(matrix)

    # Define right-hand side as a vector of ones
    b = np.ones(matrix.shape[0])

    # Extract the corresponding preconditioner for this matrix
    preconditioner_matrix = scipy.sparse.csr_matrix(prec[i])

    # Run GMRES solver
    x, exit_code = gmres(A_sparse, b, M=preconditioner_matrix, tol=1e-1, maxiter=5000)
    solutions_with_preconditioner.append(x)
    print(f"Matrix {i} solved with exit code {exit_code}")

print("All matrices solved with preconditioner.")

In [ ]:
while True:
    pass